### Setup

In [ ]:
# adapted from: Yasser H: Breast cancer logistic regression
# import necessary libraries, visualize spreadsheet in python
import pandas as pd
import numpy as np
import os
from google.colab import drive
import glob
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Find location of notebook to set correct environment
matches = glob.glob(f"/content/drive/My Drive/**/ASBME-breast-cancer-logistic-regression-key.ipynb", recursive=True)
parent = os.path.abspath(os.path.join(matches[0], os.pardir))
print(parent)
os.chdir(parent)

In [ ]:
df = pd.read_csv("./breast-cancer.csv")
df

In [ ]:
num_features = len(df.columns[2:])
print(f'Number of Features: {num_features}')
df.columns

In [ ]:
feature_1 = df['texture_mean']
feature_2 = df['radius_mean']

plt.scatter(feature_1, feature_2, c=df['malignant'], cmap='bwr')
plt.xlabel('texture_mean')
plt.ylabel('radius_mean')
plt.title('Scatter Plot visualization of two breast cancer measurements')
plt.show()

In [ ]:
# Correlation Heatmap
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

# Pairplot for first few features (adjust based on dataset size)
sns.pairplot(df.iloc[:, 1:5], hue='malignant', diag_kind='kde')
plt.show()

diagnoses = df['malignant']
feature_of_interest = df['radius_mean']
# sns.pairplot(diagnoses, hue='diagnosis')

##Sci-Kit Learn Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X, y = df.iloc[:, 2:], df['malignant']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(random_state=0, max_iter=5000).fit(X_train, y_train)
predictions = clf.predict(X_test)
print('accuracy: ', clf.score(X_test, y_test))

# print(clf.predict_proba(X_test))
# plt.bar(df.columns[2:],clf.coef_[0])
# plt.xticks(rotation=90)
# plt.show()

## Feature Engineering (Min-max Scaling)

In [ ]:
# Minmax Scaling on Features

def MinMax(feature_data):
    """
    feature_data is a vector of data for a specific feature
    Data will range from 0-1
    """

    min_val = min(feature_data)
    max_val = max(feature_data)
    range = max_val - min_val

    # epsilon prevents divide by 0 error
    epsilon = 10**(-8)

    return (feature_data - min_val)/(range + epsilon)


In [ ]:
# iloc selects all rows and only columns starting from 2nd index ('diagnosis',...)
# apply columnwise

df_scaled = df.copy()
df_scaled.iloc[:, 2:] = df.iloc[:, 2:].apply(MinMax, axis = 0)

# Check
df_scaled

## Logistic Regression

Obtain $y_{true}$ which is the vector of 0s and 1s corresponding to benign and malignant diagnosis' respectively.

In [ ]:
y_true = df_scaled['malignant']

In [ ]:
# Check ground truth y_true with diagnoses
print(y_true.shape)

print(y_true[15:22])
print(df_scaled.malignant[15:22])

Obtain $x$ which is the input dataset. We need to remove unnecessary columns from the dataframe.

In [ ]:
# Drop the id and diagnosis columns
# Obtain input dataset
df_input = df_scaled.drop(df_scaled.columns[:2], axis = 'columns')

# Check
df_input

Split the dataset into training and validation sets.
- 70% training
- 30% validation

In [ ]:
# Training and Validation Datasets
# 70% training
# 30% validation

N = len(df_input.radius_mean)
print(f'Total Number of Patients: {N}')

N_train = np.floor(N * 0.7).astype(int)
N_valid = N - N_train
print(f'Training Dataset Size: {N_train}')
print(f'Validation Dataset Size: {N_valid}')

# Split the dataset
df_train = df_input[:N_train]
df_valid = df_input[N_train:]

Objective function is the negative log likelihood. We are doing MLE - we aim to find the values of the weights that minimise the objective function. Optimisiation done via gradient descent.

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

# No bias term
# Can add a bias term by adding a feature into x that is always = 1
# Note, we are using mean NLL
def negative_log_likelihood(weights, x, y_true):
    z = np.dot(x, weights)
    phi = y_true * np.log(sigmoid(z)) + (1 - y_true) * (1 - np.log(sigmoid(z)))
    return -np.mean(phi)

def gradient(weights, x, y_true):
    z = np.dot(x, weights)
    sig_z = sigmoid(z)
    grad = np.dot(x.T, (sig_z - y_true)) / len(y_true)
    return grad

def sgd(x, y_true, weights, learning_rate, epochs):
    loss_list = []
    for epoch in range(epochs):
        grad = gradient(weights, x, y_true)

        weights -= learning_rate * grad
        if epoch % 50 == 0:
            loss = negative_log_likelihood(weights, x, y_true)
            loss_list.append(loss)
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss:.4f}')
    return weights, loss_list

In [ ]:
# Usually learning rate < 1
# Works here but not good practise
learning_rate = 1.5
epochs = 2500
init_weights = np.ones(num_features)
x = df_train.values
y_true_train = y_true[:N_train]
optimal_weights, loss_list = sgd(x, y_true_train, init_weights, learning_rate, epochs)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(len(loss_list)), loss_list)
plt.xlabel('Epochs')
plt.ylabel('Negative Log Likelihood (Loss)')
plt.title('Learning Curve')
plt.show()


Now we have obtained the optimal values of the weights, we need to check how accurate it is using the validation dataset.

In [ ]:
## Validation
def predict(x, weights):
    z = np.dot(x, weights)
    y_pred = np.where(z > 0.5, 1, 0)
    return y_pred

x_valid = df_valid.values
y_true_valid = y_true[N_train:]
y_pred = predict(x_valid, optimal_weights)

# number of correct pred / total number of pred * 100
acc = np.mean(y_pred == y_true_valid) * 100
print(f'{acc}% accuracy in validation set')

In [ ]:
# Sanity Check

print(f' Predicted: \n {y_pred[:20]}')
print(f' True: \n {y_true_valid[:20].tolist()}')

# 1 = malignant
# 0 = benign